In [1]:
import pandas as pd
import sys
import math
import imp
import pymongo as pym
import json
import numpy as np
import random
import sys
sys.path.insert(0, './lib_python/')

In [2]:
urlMongoDbLocal = "mongodb://localhost:27017";
clientLocal = pym.MongoClient(urlMongoDbLocal)

In [4]:
carSharing = clientLocal["bigDive_test"]
bookingsDB = carSharing["bookings"]
hexs = carSharing["hexs_gen"]

In [40]:
from shapely.geometry import shape
from shapely.geometry import mapping
from shapely.geometry import MultiPolygon
import folium
import seaborn as sns
from shapely.ops import cascaded_union
import json

shellIso = np.arange(0,18, 2).tolist()
shellIso.insert(0,-1)
colorIso = sns.color_palette(palette="RdBu", n_colors = len(shellIso))
colorIso = list(reversed(colorIso.as_hex()))
#colorIso = list(reversed(['#a50026','#d73027','#f46d43','#fdae61','#fee090','#e0f3f8','#abd9e9','#74add1','#4575b4','#313695']));


center = hexs.find_one({"city":city})["point"]["coordinates"]
map_folium =  folium.Map(location = [ center[1], center[0]], zoom_start=12,  tiles="Stamen Toner")

max_ = 0
count = 0
for p in hexs.find({"city":city}):
    hex_ = p["hex"]
    num_of_ends = bookingsDB.find({
        "dest_point":{
            "$geoWithin":{
                "$geometry":hex_
            }
        }        
    })
    #print(num_of_ends.count())
    diff = 0
    count_c = 0
    for car in num_of_ends:
        if car["driving_duration"] != -1:
            start_time = car["init_time"]
            end_time = car["final_time"]
            diff_b = (end_time - start_time)
            diff += diff_b - car["driving_duration"]
            count_c += 1

        
    if count_c > 0:
        diff /= (count_c * 60)
        
    if count_c > 4:
        #print(num, end="\r")
        for i, val in enumerate(shellIso):
            if diff < shellIso[i]:
                break
        def style(f):
            #print(f["geometry"]["style"]["fillColor"])
            return {
                    "fillColor" : f["geometry"]["style"]["fillColor"], 
                    "color":f["geometry"]["style"]["fillColor"],
                    "weight":0.1,
                    "fillOpacity":0.6
                   }
        hex_["style"] = {"fillColor":colorIso[i]}
        #print(i, colorIso[i])
        geojson_folium_simp = folium.GeoJson(hex_, style_function=style, )
        popup = folium.Popup('diff time {0}'.format(diff))
        popup.add_to(geojson_folium_simp)
        geojson_folium_simp.add_to(map_folium)
    count += 1
    print("count {0}".format(count), end="\r")
map_folium.save("parkings.html")
map_folium